#### Libraries

In [ ]:
# Only install the following libraries if you dont have it, otherwise leave it commented out

#!conda install -c anaconda natsort --yes
#!conda install -c anaconda xlrd --yes

#!pip install natsort --user
#!pip install xlrd --user
#!pip install pycaret[full] --user
#!pip install mlflow --user
#!pip install tune-sklearn ray[tune] --user
#!pip install optuna -- user
#!pip install hyperopt --user
#!pip install redis --user

# General Libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Sklearn Liraries
from sklearn import preprocessing

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline

import ray
from ray import tune

# Forces the print statement to show everything and not truncate
# np.set_printoptions(threshold=sys.maxsize) 
print('Libraries imported')

In [ ]:
#Receive Data
#dir_name = r'C:\Users\kswaminathan\OneDrive\01_KannaLibrary\15_Analogs'
dir_name = r'C:\Users\mkumar\Documents\GitHub\@Papers\SPE2022\Final\1_TORIS_MODEL'
filename_suffix = 'csv'

In [ ]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
df = pd.read_excel("dfssoil.xlsx", thousands=',', skiprows = skiprows)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
df.head()

In [ ]:
print(df.columns.values)

### Feature Selection

In [ ]:
#Create a copy
df_train_test_set=df.copy()

Feature = df_train_test_set[[
    'Lithology Code', 
    'Well Spacing',
    'Net Pay Pay',
    'Gross Pay',
    'Porosity', 
    'Swi',
    'Oil FVFi',
    'Temp',
    'Permeability', 
    'API Gravity', 
    'Viscosity',
    'OOIP',
    'Initial GOR',
    'Pressure Initial',
    'Fractured Faulted',
    'Shale Breaks',
    'Major Gas Cap',
    'Geologic Play',
    'Deposition System',
    'Diagenetic Overprint',
    'Structural Comp',
    'Heterogeniety',
    'Trap Type'
]]
x=Feature

y = df_train_test_set['URF'].values

print(x.head())
print(y[0:5])
print(x.shape, y.shape)

### Train-Test Split 70-30

In [ ]:
from sklearn.model_selection import train_test_split

random_state = 42
test_size = 0.3

x_train, x_test, y_train, y_test  = train_test_split(
            x, y, test_size = test_size, random_state = random_state
)

print('Train Set: ', x_train.shape, y_train.shape)
print(x_train['Permeability'][0:5])
print('Test Set: ', x_test.shape, y_test.shape)
print(x_test['Permeability'][0:5])

### Normalization as per Pycaret z-score

In [ ]:
#https://towardsdatascience.com/data-normalization-with-pandas-and-scikit-learn-7c1cc6ed6475

from sklearn.preprocessing import StandardScaler

X_train = preprocessing.StandardScaler().fit(x_train).transform(x_train)
X_test = preprocessing.StandardScaler().fit(x_test).transform(x_test)
print('Standardization X Training Set: ', X_train[0:5])
print('Standardization X Testing Set: ', X_test[0:5])

### Transformation as per Pycaret 'yeo johnson'

In [ ]:
from sklearn.preprocessing import PowerTransformer

Xt_train = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True).fit(X_train).transform(X_train)
Xt_test = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True).fit(X_test).transform(X_test)
print('Transformed X Training Set: ', Xt_train[0:5])
print('Transformed X Testing Set: ', Xt_test[0:5])

## Note that Ignore Low Variance and Remove Outliers is not implemented as it is assumed it will not make a significant difference to model

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.ensemble import RandomForestRegressor

#### K-fold and grid search CV on 2 variables only

In [ ]:
# Create a Random Forest Regressor
# win_rf=RandomForestRegressor(n_estimators=100, criterion="mse", max_depth=None, min_samples_split=2)
# win_rf.fit(Xt_train, y_train)
# yhat_rf = win_rf.predict(Xt_test)

def rfr_model(X, y, Xt):
# Perform Grid-Search
# bootstrap=True, ccp_alpha=0.0, criterion='mse',
# max_depth=None, max_features='auto', max_leaf_nodes=None,
# max_samples=None, min_impurity_decrease=0.0,
# min_impurity_split=None, min_samples_leaf=1,
# min_samples_split=2, min_weight_fraction_leaf=0.0,
# n_estimators=100, n_jobs=-1, oob_score=False,
# random_state=42, verbose=0, warm_start=False

    # Create Base Model
    rfr = RandomForestRegressor(
        bootstrap=True, ccp_alpha=0.0, criterion='mse',
        max_depth=None,
        max_features='auto', max_leaf_nodes=None,
        max_samples=None, min_impurity_decrease=0.0,
        min_impurity_split=None, min_samples_leaf=1,
        min_samples_split=2, min_weight_fraction_leaf=0.0,
        n_estimators=100, 
        n_jobs=-1, oob_score=False,
        random_state=42, verbose=0, warm_start=False
    )
    
    rfr.fit(X, y)
    predictions1 = rfr.predict(Xt)

    # Do Grid search
    gsc = GridSearchCV(
        estimator=rfr,
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=10, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    # Create K-fold Grid Model
    rfr2 = RandomForestRegressor(
        bootstrap=True, ccp_alpha=0.0, criterion='mse',
        max_depth=best_params["max_depth"],
        max_features='auto', max_leaf_nodes=None,
        max_samples=None, min_impurity_decrease=0.0,
        min_impurity_split=None, min_samples_leaf=1,
        min_samples_split=2, min_weight_fraction_leaf=0.0,
        n_estimators=best_params["n_estimators"], 
        n_jobs=-1, oob_score=False,
        random_state=42, verbose=0, warm_start=False
    )    
  
    rfr2.fit(X, y)
    # Perform K-Fold CV
    scores = cross_val_score(rfr2, X, y, cv=10, scoring='neg_mean_absolute_error')
        
    # Predict
    #predictions = cross_val_predict(rfr, X, y, cv=10)
    predictions2 = rfr2.predict(Xt)
    
    return scores, predictions1, predictions2, rfr, rfr2

In [ ]:
win_rf_grid, yhat_rf, yhat_rf_grid, rfr, rfr_grid = rfr_model(Xt_train, y_train, Xt_test)
print("MAE Score", win_rf_grid)

errors1 = abs(yhat_rf - y_test)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors1), 2), 'V/V')
mape1 = np.mean(100 * (errors1 / y_test))
accuracy1 = 100 - mape1
print('Accuracy:', round(accuracy1, 2), '%.')

errors2 = abs(yhat_rf_grid - y_test)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors2), 2), 'V/V')
mape2 = np.mean(100 * (errors2 / y_test))
accuracy2 = 100 - mape2
print('Accuracy:', round(accuracy2, 2), '%.')

In [ ]:
a = y_test
b = yhat_rf_grid
c = yhat_rf

plt.figure(figsize=(14, 8))
plt.scatter(a, b, color='blue')
plt.scatter(a, c, color='green')
plt.plot(a, a, color = 'red', label = 'x=y')
plt.xlabel("Recovery Factor (%)", size=14)
plt.ylabel("Evaluated Recovery Factor (%)", size=14)

In [ ]:
dfblind = pd.read_excel("BlindTest_SSOIL.xlsx", thousands=',', skiprows = skiprows)
#dfblind = dfblind.loc[:, ~df.columns.str.contains('^Unnamed')] 
dfblind.head()

### Blind Data Set

In [ ]:
blind_df = dfblind.copy()
blind_df.drop(['URF'], axis=1, inplace=True)

In [ ]:
blind_df_norm = preprocessing.StandardScaler().fit(blind_df).transform(blind_df)
blind_df_norm_tran = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True).fit(blind_df_norm).transform(blind_df_norm)

In [ ]:
print(dfblind['URF'])

In [ ]:
predictions = rfr.predict(blind_df_norm_tran)
print(predictions)

predictions_grid = rfr_grid.predict(blind_df_norm_tran)
print(predictions_grid)

In [ ]:
a = dfblind['URF']
b = predictions
c = predictions_grid

plt.figure(figsize=(14, 8))
plt.scatter(a, b, color='blue')
plt.scatter(a, c, color='green')
plt.plot(a, a, color = 'red', label = 'x=y')
plt.xlabel("Recovery Factor (%)", size=14)
plt.ylabel("Evaluated Recovery Factor (%)", size=14)